In [1]:
# importing the needed package 
import csv
# it assumed the file will be loacted in the same folder as this notebook
filename = "groceries.csv"
# Thershold value is declared as a global variable
support =300


In [2]:
#class FPNode describing its properties 
class FPNode:
    def __init__(self, itemName,frequency,parentNode):
        self.name = itemName
        self.frequency = frequency
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
# This function increases the frequency based on the transaction      
    def inc(self, counter):
        self.frequency += counter
# This function prints tree recursively and returms list    
    def printtree(self):
        result = []
        if len(self.children)==0:
            return [self.name + " " + str(self.frequency)]
        for child in self.children.values():
            result += child.printtree()
        return [self.name + " " + str(self.frequency), result]

In [3]:
##########################################################################################################
            #This part updates the nodelink of the node with the incoming children    
##########################################################################################################
    
def updateNodeLink(items, inTree, headertable,count):
#check if the item is in children node
    if (items[0] in inTree.children):
        exists = 1 
    else:
        exists = 0
#If it exists increase the count else create the children node        
    if exists:
        inTree.children[items[0]].inc(count)
    else:
        inTree.children[items[0]] = FPNode(items[0], count, inTree)
       
    if (headertable[items[0]][1] == None): #Check if the header Node link already exists
        headertable[items[0]][1] = inTree.children[items[0]] #Update the first pointer with Current
        # Update the last pointer as well which will be used later to update the incoming
        headertable[items[0]][2] = inTree.children[items[0]] 
    else:
        if not exists:# if the first pointer is not none then update the nodelink of the pointer stored in second one with pointer of the incoming children node
            headertable[items[0]][2].nodeLink= inTree.children[items[0]]
            headertable[items[0]][2]= inTree.children[items[0]]

    if len(items) > 1:#recursively call the updateNodeLink function on remaining items
        updateNodeLink(items[1::], inTree.children[items[0]], headertable, count)
    

In [4]:
def FPtreecreation():

##########################################################################################################   
                                        #first scan through the dataset
##########################################################################################################
    headertable ={}    
    with open(filename) as f: # opening the csv file
        transactions = csv.reader(f, delimiter=",")# reading the csv file
        for transaction in transactions:# Looping over every transactions in the file
            transaction = list(filter(None, transaction)) # Removing the null values in the csv file
            for item in transaction:# Looping over every item in each transaction
                headertable[item]=1 if headertable.get(item,0)==0 else headertable[item]+1 # updating the count of each item      
    
    # Keeping items with support greater than 300
    headertable= {k:v for k ,v in headertable.items() if v >=support}
    
    #sorting the headetable based on count in descending order 
    sorted_d = sorted(headertable.items(), key=lambda x: x[1], reverse=True)
    
    # Adding two pointers to the headertable. The first one is to store the location of the first item of each category
    #second one is to store the upcoming item of that category
    headertable={k:[v, None,None] for k ,v in sorted_d}
    
    #Creates a node for the fp tree
    root = FPNode('Root', 1, None)
##########################################################################################################
                                        #Second scan through the dataset
##########################################################################################################
    
    with open(filename) as f: # opening the csv file
        reader = csv.reader(f, delimiter=",")# reading the csv file
        for i, line in enumerate(reader):# Looping over every transactions in the file
            line = list(filter(None, line)) # Removing the null values in the csv file
            ordereditem = {}#creating an empty dictionary
            for item in line:
                if item in headertable: ordereditem[item]=headertable[item][0]
            if len(ordereditem) > 0: 
                #sorting the items alphabetically if they have same count
                Freqordereditem = [v[0] for v in sorted(ordereditem.items(),key =lambda p: p[1], reverse=True)]
                updateNodeLink(Freqordereditem, root, headertable, 1)#updating the nodelinks to generate the Fp Tree
    return root, headertable
    
        

In [5]:
#This function creates a conditional Fp Tree. This function is more or less as
# the FPTree
def condFPTree(dataSet):
    condheaderTable = {}
    for trans in dataSet:
        for item in trans:
            condheaderTable[item] = condheaderTable.get(item, 0) + dataSet[trans]
    
    #remove items which are below support levels
    for k in list(condheaderTable.keys()):
        if condheaderTable[k] < support:
            del(condheaderTable[k])
      
    for k in condheaderTable:
        condheaderTable[k] = [condheaderTable[k], None, None]
    freqItemSet = list(set(condheaderTable.keys()))
    
    #sort the items
    condheaderTable=sorted(sorted(condheaderTable.items(), key = lambda x : x[0]), key = lambda x : x[1], reverse=True)
    condheaderTable=dict(condheaderTable)
 
    retTree = FPNode('Root', 1, None)
    
    for tranSet, count in dataSet.items():
        localD = {}
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = condheaderTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(),
                            key=lambda p: p[1], reverse=True)]
            updateNodeLink(orderedItems, retTree, condheaderTable, count)
    return retTree, condheaderTable

In [6]:
#checks for the parent of the treeNode pointer if yes it appends the name of the parent in the previtem and recursively 
#calls itself until the root node is reached
def climbTree(LNode, previtem): 
    if LNode.parent != None:
        previtem.append(LNode.name)
        climbTree(LNode.parent, previtem)

def conditionalpath(basepath, treeNode):
    condpaths = {}
    #checks the treeNode pointer to have any value if yes call the climbtree function
    while treeNode != None:
        previtem = []
        climbTree(treeNode, previtem) 
        #after reaching the root node in climb tree it comes here to check if length of previtem is greater than one
        if len(previtem) > 1: 
            #to iterate through the list it is converted as frozen set
            condpaths[frozenset(previtem[1:])] = treeNode.frequency
        treeNode = treeNode.nodeLink
    return condpaths


     

In [7]:
def genFreqitems(headertable, prefix, freqitemlist): # creates a conditional fp-tree
    #sorts the items in ascending order
    Q2b = csv.writer(open("Q2_b.csv", 'w'))
    sort_headertable = [b[0] for b in sorted(headertable.items(), key = lambda val: val[1][0])]
    #print(len(headertable))
    #append each item to a list
    for item in sort_headertable:
        newfreqset = prefix.copy()
        newfreqset.add(item)
        freqitemlist.append(newfreqset)
#calling conditionalpath function to build the conditional patterbase
#checks for the length of condpatternbase to be zero if not it calls the condFPTree fucn. to create the Conditional FP tree
#for the particular item. And recursivley calls the genFreqitems function until conditional FP tree is built for all 
#items in the headertable 

        condpatternbase = conditionalpath(item, headertable[item][1])#calling conditionalpath function to build the conditional patterbase
        if len(condpatternbase)>0:
            #print(len(condpatternbase))
            mycondtree, condheadertable = condFPTree(condpatternbase) 
            if len(condheadertable) >0: 
                
                cfptreeList=mycondtree.printtree()
                Q2b.writerow(cfptreeList)
                print('CFP Tree (' + item  + '):  ' + str(cfptreeList) )
            genFreqitems(condheadertable, newfreqset, freqitemlist)

In [8]:
FPtree, HeaderTable = FPtreecreation()
frequent_itemset =[]
print("Answer of Q2_b")
genFreqitems(HeaderTable,  set([]), frequent_itemset)


Answer of Q2_b
CFP Tree (whipped/sour cream):  ['Root 1', ['whole milk 317']]
CFP Tree (citrus fruit):  ['Root 1', ['whole milk 300']]
CFP Tree (pastry):  ['Root 1', ['whole milk 327']]
CFP Tree (sausage):  ['Root 1', ['rolls/buns 301']]
CFP Tree (tropical fruit):  ['Root 1', ['other vegetables 185', 'whole milk 416', ['other vegetables 168']]]
CFP Tree (root vegetables):  ['Root 1', ['other vegetables 238', 'whole milk 481', ['other vegetables 228']]]
CFP Tree (bottled water):  ['Root 1', ['whole milk 338']]
CFP Tree (yogurt):  ['Root 1', ['whole milk 551', ['other vegetables 219', ['rolls/buns 59'], 'rolls/buns 94'], 'rolls/buns 131', 'other vegetables 208', ['rolls/buns 54']]]
CFP Tree (soda):  ['Root 1', ['rolls/buns 290', ['other vegetables 54'], 'whole milk 394', ['other vegetables 94', 'rolls/buns 87', ['other vegetables 43']], 'other vegetables 131']]
CFP Tree (rolls/buns):  ['Root 1', ['other vegetables 243', 'whole milk 557', ['other vegetables 176']]]
CFP Tree (other vegetab

In [9]:
print("Answer of Q2_a")
# writing the csv file to current location
writer = csv.writer(open("Q2_a.csv", 'w'))
for i in frequent_itemset:
    print(i)
    writer.writerow(set(i))

Answer of Q2_a
{'onions'}
{'hygiene articles'}
{'hamburger meat'}
{'berries'}
{'UHT-milk'}
{'sugar'}
{'dessert'}
{'long life bakery product'}
{'salty snack'}
{'waffles'}
{'cream cheese '}
{'white bread'}
{'chicken'}
{'frozen vegetables'}
{'chocolate'}
{'napkins'}
{'beef'}
{'curd'}
{'butter'}
{'pork'}
{'coffee'}
{'margarine'}
{'frankfurter'}
{'domestic eggs'}
{'brown bread'}
{'whipped/sour cream'}
{'whipped/sour cream', 'whole milk'}
{'fruit/vegetable juice'}
{'pip fruit'}
{'canned beer'}
{'newspapers'}
{'bottled beer'}
{'citrus fruit'}
{'citrus fruit', 'whole milk'}
{'pastry'}
{'pastry', 'whole milk'}
{'sausage'}
{'rolls/buns', 'sausage'}
{'shopping bags'}
{'tropical fruit'}
{'other vegetables', 'tropical fruit'}
{'tropical fruit', 'whole milk'}
{'root vegetables'}
{'root vegetables', 'other vegetables'}
{'root vegetables', 'whole milk'}
{'bottled water'}
{'bottled water', 'whole milk'}
{'yogurt'}
{'rolls/buns', 'yogurt'}
{'yogurt', 'other vegetables'}
{'yogurt', 'whole milk'}
{'soda'}